In [ ]:
import os
from diffaux.validation.read_size_validation_data import read_size_data, validation_info
from diffaux.validation.read_size_validation_data import get_author_list

##  Get Validation Data and Plot Behavior

In [ ]:
authors_rm = get_author_list(validation_info['Re_vs_Mstar'])
authors_re = get_author_list(validation_info['Re_vs_z_data'])

In [ ]:
Redata = read_size_data({}, authors_re, info_key='Re_vs_z_data')

In [ ]:
Rmdata= read_size_data({}, authors_rm, info_key='Re_vs_Mstar')

In [ ]:
authors = get_author_list(validation_info['Re_vs_z'])
data= read_size_data({}, authors, info_key='Re_vs_z')

In [ ]:
authors_rd = get_author_list(validation_info['Re_vs_Mstar_data'])
Rddata = {}
for author in authors_rd:
    samples = validation_info['Re_vs_Mstar_data'][author]['samples']
    for sample in samples:
        if 'All' not in sample:
            Rddata = read_size_data(Rddata, authors_rd, info_key='Re_vs_Mstar_data', sample=sample)


In [ ]:
notebook_dir = '/Users/kovacs/cosmology/diff_notebooks'
from diffaux.validation.plot_size_data import plot_size_data
from diffaux.validation.plot_size_data import PLOT_DRN

In [ ]:
plot_size_data(Rddata, validation_info, authors, info_keys=['Re_vs_Mstar_data'], plotdir=os.path.join(notebook_dir, PLOT_DRN))

In [ ]:
plot_size_data(Redata, validation_info, authors, info_keys=['Re_vs_z_data'], plotdir=os.path.join(notebook_dir, PLOT_DRN), summary_only=True)

### Plot Behavior of Fit Coefficients for Median Sizes

In [ ]:
plot_size_data(Rmdata, validation_info, authors_rm, info_keys=['Re_vs_Mstar'], summary_only=True, plotdir=os.path.join(notebook_dir, PLOT_DRN))

In [ ]:
plot_size_data(data, validation_info, authors, info_keys=['Re_vs_z'], plotdir=os.path.join(notebook_dir, PLOT_DRN), summary_only=True)

## Setup Choices for Fitting the Behavior of Median Sizes with z

In [ ]:
from diffaux.size_modeling.fit_size_data import Samples_zFit
from diffaux.size_modeling.fit_size_data import zFitParams_initial
print(zFitParams_initial)
print(Samples_zFit)

In [ ]:
from diffaux.size_modeling.fit_size_data import collect_data_vectors
#authors_fit = get_author_list(validation_info['Re_vs_z'], lambda_min=0.5, lambda_max=1.0)
#data_fit = read_size_data({}, authors_fit, info_key='Re_vs_z')
data_vectors = collect_data_vectors(data, Samples_zFit, validation_info, fit_type='Re_vs_z',
                                    lambda_min=0.5, lambda_max=1.0)

In [ ]:
#print(data_vectors)
from diffaux.size_modeling.fit_size_data import assign_p0_values_to_fits
#assign initial values to fits dict and plot
fits = assign_p0_values_to_fits(zFitParams_initial, fit_type='Re_vs_z')

In [ ]:
from diffaux.validation.plot_size_fits import plot_fits

### Plot Initial Guess for Fit Results and Compare to Data

In [ ]:
fit_type = 'Re_vs_z'
plot_fits(fits[fit_type], data_vectors[fit_type], Samples_zFit, validation_info[fit_type], plotdir=os.path.join(notebook_dir, PLOT_DRN),
          title='_InitialGuess')

## Fit the Coefficients and Save Results

In [ ]:
fit_type = 'Re_vs_z'
from diffaux.size_modeling.fit_size_data import fit_parameters, _sigmoid
fits = {}
from diffaux.size_modeling.fit_size_data import Xvalue_zFit
print(zFitParams_initial)
fits[fit_type] = fit_parameters(data_vectors[fit_type], Xvalue_zFit, zFitParams_initial, func=_sigmoid)

In [ ]:
#print(fits)
author_list = get_author_list(validation_info['Re_vs_z'], lambda_min=0.5, lambda_max=1.0)
plot_fits(fits[fit_type], data_vectors[fit_type], Samples_zFit, validation_info[fit_type], func=_sigmoid,
          data=data[fit_type], authors=author_list, plotdir=os.path.join(notebook_dir, PLOT_DRN))


In [ ]:
import pickle
from diffaux.size_modeling.fit_size_data import zFitParameters
from diffaux.size_modeling.fit_size_data import write_fit_parameters, read_fit_parameters

In [ ]:
#write_fit_parameters(fits, fitdir=os.path.join(fit_dir, FIT_DRN))
write_fit_parameters(fits, fitdir='./')

## Read in Fit Parameters and Generate Sizes

In [ ]:
fit_pars, fits = read_fit_parameters(zFitParameters)
print(fit_pars)

In [ ]:
#generate sizes
import numpy as np
from diffaux.size_modeling.fit_size_data import generate_sizes, get_color_mask, _sigmoid, median_size_vs_z
fit_func=_sigmoid
size_func=median_size_vs_z
N = 5000
lM_lo = 9.0
lM_hi = 12.0
z_lo = 0.
z_hi = 3.0
log_Mstar = np.random.uniform(low=lM_lo, high=lM_hi, size=N)
redshift = np.random.uniform(low=z_lo, high=z_hi, size=N)
color_gal = np.random.uniform(low=-0.2, high=2.3, size=N)
#print(log_Mstar, redshift, color)

In [ ]:
Re, R_med, scatter_up, scatter_down = generate_sizes(zFitParams_initial, log_Mstar, redshift, color_gal,
                                                     samples=Samples_zFit)
print(np.min(R_med), np.max(R_med))
print(np.min(Re), np.max(Re))

## Plot Results and Compare Generated Sizes with Validation Data

In [ ]:
tests = [Redata, Rddata]

In [ ]:
# make validation plots using data for measured sizes and median sizes
from diffaux.validation.plot_size_fits import plot_generated_sizes
for test, ikey in zip(tests, ['Re_vs_z_data', 'Re_vs_Mstar_data']):
    authors = get_author_list(validation_info[ikey])
    plot_generated_sizes(Re, R_med, color_gal, log_Mstar, redshift, Samples_zFit, 
                         authors, test[ikey], validation_info[ikey],
                         plotdir=os.path.join(notebook_dir, PLOT_DRN))


In [ ]:
#test behavior of R_med
from diffaux.validation.plot_size_fits import plot_Re_median

In [ ]:
plot_Re_median(fit_pars, Samples_zFit, plotdir=os.path.join(notebook_dir, PLOT_DRN))

### Run test for generated sizes: 

In [ ]:
#Test use facing functions
from diffaux.size_modeling.tests.test_sizes import test_generate_sizes

In [ ]:
#Run the test in write mode to generate test data
test_generate_sizes(read=False, testdata_dir='./')

In [ ]:
#Run the test in read mode to read test data and compare with tthe user facing function
test_generate_sizes(read=True)